In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from collections import OrderedDict
from math import ceil, floor
from operator import itemgetter

## Scrape Wikipedia Page

In [2]:
# list of state abbreviations
states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL',
          'GA','HI','ID','IL','IN','IA','KS','KY','LA','ME',
          'MD','MA','MI','MN','MS','MO','MT','NE','NV','NH',
          'NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
          'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY', 'US']

def append_sum(ec):
    ec.append(sum(ec))

ec_votes_2000=[9,  3,  8,  6,  54, 8,  8,  3,  3,  25,
          13, 4,  4,  22, 12, 7,  6,  8,  9,  4,
          10, 12, 18, 10, 7,  11, 3,  5,  4,  4,
          15, 5,  33, 14, 3,  21, 8,  7,  23, 4,
          8,  3,  11, 32, 5,  3,  13, 11, 5,  11, 3]
append_sum(ec_votes_2000)
ec_votes_2004=[9,  3,  10, 6,  55, 9,  7,  3,  3,  27,
               15, 4,  4,  21, 11, 7,  6,  8,  9,  4,
               10, 12, 17, 10, 6,  11, 3,  5,  5,  4,
               15, 5,  31, 15, 3,  20, 7,  7,  21, 4,
               8,  3,  11, 34, 5,  3,  13, 11, 5,  10, 3]
append_sum(ec_votes_2004)
ec_votes_2012=[9,  3,  11, 6,  55, 9,  7,  3,  3,  29,
          16, 4,  4,  20, 11, 6,  6,  8,  8,  4,
          10, 11, 16, 10, 6,  10, 3,  5,  6,  4,
          14, 5,  29, 15, 3,  18, 7,  7,  20, 4,
          9,  3,  11, 38, 6,  3,  13, 12, 5,  10, 3]
append_sum(ec_votes_2012)
ec_votes = list(zip(ec_votes_2000, ec_votes_2004, ec_votes_2012))
ec_votes_dict = OrderedDict()
for i in range(len(states)):
    ec_votes_dict[states[i]] = ec_votes[i]
ec_votes = pd.DataFrame.from_dict(ec_votes_dict, orient='index')
ec_votes.rename(columns={0: '2000', 1: '2004', 2: '2012'}, inplace=True)
ec_votes['2016']=ec_votes['2012']
ec_votes['2008']=ec_votes['2004']
ec_votes = ec_votes.sort_index(axis=1)

In [3]:
ec_votes.tail()

,2000,2004,2008,2012,2016
WA,11,11,11,12,12
WV,5,5,5,5,5
WI,11,10,10,10,10
WY,3,3,3,3,3
US,538,538,538,538,538


In [4]:
def clean_and_convert(count_text):
    clean = count_text.strip().replace(',','').replace('.', '').replace('-','0').replace(
                'N/A', '0').replace('★', '').replace('–', '')
    return int(clean) if len(clean) else 0

In [5]:
def get_table_rows(year, num_candidates):
    page = BeautifulSoup(urllib.request.urlopen(
            'https://en.wikipedia.org/wiki/United_States_presidential_election,_'+year).read(), "html.parser")
    header = page.find('span', id='Results_by_state').parent
    table_container = header.find_next_sibling('div')
    table = table_container.table
    if table is None:
        table_container = table_container.find_next_sibling('div')
        table = table_container.table
    head_row = table.find('tr')
    all_rows = list(table.find_all('tr'))
    vote_rows = all_rows[2:]
#     last_row = all_rows[-1]
    return get_headings(head_row, num_candidates), vote_rows


def get_headings(head_row, num_candidates):
    headings = []
    for cell in head_row.find_all('th', attrs={'colspan': '3'}):
        children = list(cell.children)
        if len(children) == 3:
            headings.append((children[0].strip(), children[2].strip()))
        else:
            headings.append((children[0], None))
    headings = headings[:num_candidates]
    headings.append(('Total', None))
    return headings


def read_in_data(year, num_candidates, total_column):
    """
    year - string containing 4-digit election year
    num_candidates - the number of candidate columns in the "Results_by_state" table, including "Other"
    total_column - the column number that contains the total votes for each state/district, counting from zero
    returns: a Pandas DataFrame, indexed by state/district, with columns for each candidate, total vote, and
             state total electoral college votes
    """
    headings, vote_rows = get_table_rows(year, num_candidates)
    data = OrderedDict()
    columns_with_vote_counts = list(range(2, 3*num_candidates, 3))
    columns_with_vote_counts.append(total_column)
    abbr_column = total_column + 1
    vote_rows = list(zip(vote_rows, ['td'] * len(vote_rows)))
    vote_rows[-1] = (vote_rows[-1][0], 'th') # last row uses <th> instead of <td>
    for row, cell_type in vote_rows:
        cells = list(row.find_all(cell_type))
        abbr = cells[abbr_column].string.strip()
        if len(abbr)==2 or len(abbr)==6:
            abbr = abbr[:2]
            data[abbr] = [clean_and_convert(cells[i].text) for i in columns_with_vote_counts]
    abbreviations  = []
    for h in headings:
        abbreviations.append(''.join([s[0] for s in h[0].split()]))
    vote_data = pd.DataFrame.from_dict(data, orient='index')
    vote_data.columns=abbreviations
    vote_data['E'] = ec_votes[year]
    return vote_data

In [6]:
vote_data_2000 = read_in_data('2000', 8, 28)

In [7]:
vote_data_2004 = read_in_data('2004', 7, 25)

In [8]:
vote_data_2008 = read_in_data('2008', 7, 25)

In [9]:
vote_data_2012 = read_in_data('2012', 5, 19)

In [10]:
vote_data_2016 = read_in_data('2016', 6, 20)

In [11]:
vote_data_2000.tail()

,GWB,AG,RN,PB,HB,HP,JH,O,T,E
WA,1108864,1247652,103002,7171,13135,1989,2927,2693,2487433,11
WV,336475,295497,10680,3169,1912,23,367,1,648124,5
WI,1237279,1242987,94070,11471,6640,2042,853,3265,2598607,11
WY,147947,60481,4625,2724,1443,720,411,0,218351,3
US,50456002,50999897,2882955,448895,384431,98020,83714,51186,105405100,538


In [12]:
vote_data_2004.tail()

,GWB,JK,RN,MB,MP,DC,O,T,E
WA,1304894,1510201,23283,11955,3922,2974,1855,2859084,11
WV,423778,326541,4063,1405,82,5,13,755887,5
WI,1478120,1489504,16390,6464,0,2661,3868,2997007,10
WY,167629,70776,2741,1171,631,0,480,243428,3
US,62040610,59028444,465151,397265,143630,119859,99887,122294846,538


In [13]:
vote_data_2008.tail()

,BO,JM,RN,BB,CB,CM,O,T,E
WA,1750848,1229216,29489,12728,9432,3819,1346,3036878,11
WV,303857,397466,7219,0,2465,2355,89,713451,5
WI,1677211,1262393,17605,8858,5072,4216,8062,2983417,10
WY,82868,164958,2525,1594,1192,0,1521,254658,3
US,69498516,59948323,739034,523715,199750,161797,242685,131313820,538


In [14]:
vote_data_2012.tail()

,BO,MR,GJ,JS,O,T,E
WA,1755396,1290670,42202,20928,16320,3125516,12
WV,238269,417655,6302,4406,3806,670438,5
WI,1620985,1407966,20439,7665,11379,3068434,10
WY,69286,170962,5326,0,3487,249061,3
US,65915795,60933504,1275971,469627,490512,129085409,538


In [15]:
vote_data_2016.tail()

,HC,DT,GJ,JS,EM,O,T,E
WA,1742718,1221747,160879,58417,0,25453,3209214,12
WV,188794,489371,23004,8075,0,3807,713051,5
WI,1382536,1405284,106674,31072,11855,38729,2976150,10
WY,55973,174419,13287,2515,0,9655,255849,3
US,65853625,62985105,4489233,1457222,728860,1579672,137098443,538


## Export data

In [16]:
def export_data(year, df):
    filename = 'state-vote-data-{}.csv'.format(year)
    df.to_csv(filename, sep=',')
    
export_data('2000', vote_data_2000)
export_data('2004', vote_data_2000)
export_data('2008', vote_data_2000)
export_data('2012', vote_data_2012)
export_data('2016', vote_data_2016)

In [17]:
def calculate_fair_efficient_ec_votes(df):
    ec_votes = OrderedDict()
    wasted = OrderedDict()
    for i in range(51):
        st = df.iloc[i]
        c = OrderedDict(st.loc[:'O'])
        e = OrderedDict()
        r = {}
        E = st.loc['E']
        V = st.loc['T']
        if V == 0:
            V = sum(list(st)[:6])
        for candidate, pop_votes in c.items():
            e[candidate] = floor(E * pop_votes / V)
            r[candidate] = ceil(pop_votes - V * e[candidate] / E)
        r = OrderedDict(sorted(r.items(), key=itemgetter(1), reverse=True))
        remainder = E - sum(e.values())
        for candidate in r.keys():
            if candidate != 'O': # not mappable to a single candidate
                e[candidate] += 1
                remainder -= 1
                r[candidate] = 0
                if remainder == 0:
                    break
        ec_votes[st.name] = e
        wasted[st.name] = r
    ec_votes = pd.DataFrame.from_dict(ec_votes, orient='index')
    wasted = pd.DataFrame.from_dict(wasted, orient='index')
    return ec_votes, wasted
    
def print_fair_ec_vote_summary(year, df, show_state_data=False):
    print(year)
    print('====')
    ec_votes, wasted = calculate_fair_efficient_ec_votes(df)
    if show_state_data:
        print('Electoral College Votes by State:\n{}\n'.format(ec_votes))
        print('Wasted Popular Votes by State:\n{}\n'.format(wasted))
    ecsum = ec_votes.sum()
    print('Electoral College Tally:\n{}\nTotal:{}\n'.format(ecsum, ecsum.sum()))
    wsum = wasted.sum()
    print('Wasted popular votes:\n{}\nTotal: {}'.format(wsum, wsum.sum()))
    print('')
    print()
    
print_fair_ec_vote_summary('2000', vote_data_2000)
print_fair_ec_vote_summary('2004', vote_data_2004)
print_fair_ec_vote_summary('2008', vote_data_2008)
print_fair_ec_vote_summary('2012', vote_data_2012)
print_fair_ec_vote_summary('2016', vote_data_2016)

2000
====
Electoral College Tally:
GWB    263
AG     262
RN      13
PB       0
HB       0
HP       0
JH       0
O        0
dtype: int64
Total:538

Wasted popular votes:
AG      863577
RN     1197844
GWB     765498
PB      448895
HB      384431
HP       98020
O        51186
JH       83714
dtype: int64
Total: 3893165


2004
====
Electoral College Tally:
GWB    280
JK     258
RN       0
MB       0
MP       0
DC       0
O        0
dtype: int64
Total:538

Wasted popular votes:
GWB     858574
JK     1175528
RN      465151
MB      397265
MP      143630
O        99887
DC      119859
dtype: int64
Total: 3259894


2008
====
Electoral College Tally:
BO    289
JM    248
RN      1
BB      0
CB      0
CM      0
O       0
dtype: int64
Total:538

Wasted popular votes:
BO     979528
JM    1473774
RN     630653
BB     523715
CB     199750
O      242685
CM     161797
dtype: int64
Total: 4211902


2012
====
Electoral College Tally:
BO    276
MR    261
GJ      1
JS      0
O       0
dtype: int64
Total:538

